**Regras de associação**

**Algoritmo Apriori**

**Veja: https://www.youtube.com/watch?v=0v6PiOpsJi8**

In [ ]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=e8e9890cca4045c8454de399003f15d0045dd4e2a323cb60c5558476e1bbbd83
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [ ]:
import pandas as pd
from apyori import apriori

In [ ]:
#encoding = "cp1252", encoding='utf-8', encoding='latin1'etc
base= pd.read_csv('/content/sample_data/Aula25-10.csv', sep=';',encoding='cp1252', header=None)
base

0           1      2      3       4
0     NaN  Detergente  Limao    NaN  Feijão
1  Alcool  Detergente  Limao    NaN     NaN
2  Alcool  Detergente    NaN  arroz     NaN
3     NaN         NaN  Limao    NaN     NaN
4     NaN         NaN  Limao    NaN     NaN
5  Alcool         NaN  Limao  arroz  Feijão
6  Alcool  Detergente  Limao    NaN     NaN

In [ ]:
base.shape

(7, 5)

**Como estaremos treinando um modelo a priori, que recebe entradas em formato de lista, precisamos transformar o dataframe em uma lista de transações.**

In [ ]:
transacoes = []
for i in range(len(base)):
  #print(i)
  #print(base_mercado1.values[i, 0])
  transacoes.append([str(base.values[i, j]) for j in range(base.shape[1])])

In [ ]:
transacoes

[['nan', 'Detergente', 'Limao', 'nan', 'Feijão'],
 ['Alcool', 'Detergente', 'Limao', 'nan', 'nan'],
 ['Alcool', 'Detergente', 'nan', 'arroz', 'nan'],
 ['nan', 'nan', 'Limao', 'nan', 'nan'],
 ['nan', 'nan', 'Limao', 'nan', 'nan'],
 ['Alcool', 'nan', 'Limao', 'arroz', 'Feijão'],
 ['Alcool', 'Detergente', 'Limao', 'nan', 'nan']]

In [ ]:
type(transacoes)

list

**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [ ]:
regras = apriori(transacoes, min_support = 0.3, min_confidence = 0.6)
saida = list(regras)
print(len(saida))
print(saida)

11
[RelationRecord(items=frozenset({'Limao'}), support=0.8571428571428571, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Limao'}), confidence=0.8571428571428571, lift=1.0)]), RelationRecord(items=frozenset({'nan'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0)]), RelationRecord(items=frozenset({'Alcool', 'Detergente'}), support=0.42857142857142855, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Alcool'}), items_add=frozenset({'Detergente'}), confidence=0.75, lift=1.3125), OrderedStatistic(items_base=frozenset({'Detergente'}), items_add=frozenset({'Alcool'}), confidence=0.75, lift=1.3125)]), RelationRecord(items=frozenset({'Limao', 'Alcool'}), support=0.42857142857142855, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Alcool'}), items_add=frozenset({'Limao'}), confidence=0.75, lift=0.875)]), RelationRecord(items=frozenset({'nan', 'Alcool'}),

In [ ]:
print(saida[0])

RelationRecord(items=frozenset({'Limao'}), support=0.8571428571428571, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Limao'}), confidence=0.8571428571428571, lift=1.0)])


In [ ]:
print(saida[1])

RelationRecord(items=frozenset({'nan'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0)])


In [ ]:
print(saida[2])

RelationRecord(items=frozenset({'Alcool', 'Detergente'}), support=0.42857142857142855, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Alcool'}), items_add=frozenset({'Detergente'}), confidence=0.75, lift=1.3125), OrderedStatistic(items_base=frozenset({'Detergente'}), items_add=frozenset({'Alcool'}), confidence=0.75, lift=1.3125)])


**Para facilitar a visualização, veja como transformar isso em um dataframe**

Extraído de:

https://www.section.io/engineering-education/apriori-algorithm-in-python/

https://splunktool.com/understanding-apyoris-output

In [ ]:
Antecedente = []
Consequente = []
suporte = []
confianca = []
lift = []
RegrasFinais = []

for resultado in saida:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    suporte.append(s)
    confianca.append(c)
    lift.append(l)
    RegrasFinais = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [ ]:
RegrasFinais

Antecedente   Consequente   suporte  confianca    lift
0      [Alcool]  [Detergente]  0.428571       0.75  1.3125
1  [Detergente]      [Alcool]  0.428571       0.75  1.3125
2      [Alcool]       [Limao]  0.428571       0.75  0.8750
3  [Detergente]       [Limao]  0.428571       0.75  0.8750

**Podemos ordenar estas regras por uma métrica desejada**

In [ ]:
RegrasFinais.sort_values(by='lift', ascending =False)

Antecedente   Consequente   suporte  confianca    lift
0      [Alcool]  [Detergente]  0.428571       0.75  1.3125
1  [Detergente]      [Alcool]  0.428571       0.75  1.3125
2      [Alcool]       [Limao]  0.428571       0.75  0.8750
3  [Detergente]       [Limao]  0.428571       0.75  0.8750